In [1]:
# requirements
# pip install pims_nd2 tifffile numpy
import tifffile as tf
import numpy as np
from pims import ND2_Reader
import time

In [3]:
# user parameters
fname = '/users/sarah/Documents/data/data_wb/20190116_SKN/20190116_SKN_FC036_naive_111nmBtn_40xWI.nd2'
num_z = 11

In [4]:
# begin code
if not fname.endswith('.nd2'):
    print('Filename should end with .nd2 !!! Exiting...')
    exit()

In [5]:
print('Converting ND2 file to tiff!')

# load nd2 file
frames = ND2_Reader(fname)

Converting ND2 file to tiff!


/Users/sarah/anaconda3/lib/python3.7/site-packages/pims/base_frames.py:522: UserWarning: Please call FramesSequenceND.__init__() at the start of thethe reader initialization.
  warn("Please call FramesSequenceND.__init__() at the start of the"


In [6]:
# check if nd2 file already has z planes specified
sizes = frames.sizes
if 'z' in sizes:
    print('Dataset is already organized with z = {}'.format(sizes['z']))
    if sizes['z'] != num_z:
        print('Dataset z does not match input z! Exiting :(')
        exit()

In [7]:
print(frames.sizes)

{'x': 600, 'y': 200, 't': 54890}


In [8]:
# guess at number of 2gb files we want to output
# nb: data should be in 16-bit
if frames.pixel_type != np.uint16:
    print('Pixel data type not recognized! Currently only supports uint16')

In [16]:
# xyz * 16 bit / 8 bits per byte
# means to check how much space (in bytes) a volume takes up
# fixed else: statement to calculate appropriately
volume_byte_size = sizes['x'] * sizes['y'] * num_z * 16 / 8

# 2 billion bytes is 2gb
vols_per_file = int(np.floor(2000000000 / volume_byte_size))
if 'z' in sizes:
    total_files_to_output = int(np.ceil(sizes['t']/vols_per_file))
else:
    total_files_to_output = int(np.ceil((sizes['t']/ num_z) / vols_per_file))

In [17]:
# check
print(volume_byte_size)

2640000.0


In [18]:
print(vols_per_file)

757


In [19]:
'z' in sizes

False

In [20]:
print(total_files_to_output)

7


In [21]:
# set total number of volumes to write
if 'z' not in sizes:
    if sizes['t'] % num_z != 0:
        print('Number of frames {} is not evenly divisible by {}. Clipping last frames...'.format(sizes['t'], num_z))
    vols_to_write = sizes['t'] // num_z

else:
    vols_to_write = sizes['t']

In [22]:
print(vols_to_write)

4990


In [23]:
# start writing volumes to tiff file
output_fname_base = fname[:-4] + '_converted_'
vol_counter = 0
file_counter = 0
t0 = time.time()

In [24]:
for output_ndx in range(total_files_to_output):
    print(output_ndx)

0
1
2
3
4
5
6


In [25]:
# by setting output_ndx to 0, we can loop through once.
output_ndx = 0

In [27]:
# set output fname with padded zeros
tw = tf.TiffWriter(output_fname_base + str(file_counter).zfill(4) + '.tiff', bigtiff=True)

In [1]:
# assemble and write volumes
v = 0

In [30]:
# helpful print statement on progress
if vol_counter % 100 == 0:
    print('Writing volume {} of {}'.format(vol_counter, vols_to_write))

Writing volume 0 of 4990


In [2]:
# assemble a volume from individual frames

if 'z' not in sizes:
    vol = np.zeros((num_z, sizes['y'], sizes['x']))
    for f in range(num_z):
        frame_ndx = vol_counter * num_z + f
        vol[f, :, :] = frames[frame_ndx]

NameError: name 'sizes' is not defined